In [1]:
import math
import mysql.connector as mysql
import numpy as np
import os
import pandas as pd
import re
import urllib.request

from os.path import splitext
from pathlib import Path
from PIL import ImageFile

In [2]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

# == Files config ==
FILE_FOLDER = r"D:\legal_data\amazon_2020.csv"
fdata=pd.read_csv(Path(FILE_FOLDER))
IMG_FOLDER_DEST = r"D:\images\_amazon"

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_amazon"

In [3]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

cursor.execute("DROP TABLE IF EXISTS products_tmp_amazon")

In [4]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_amazon ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 'f'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [5]:
fdata.columns

Index(['Uniq Id', 'Product Name', 'Brand Name', 'Asin', 'Category',
       'Upc Ean Code', 'List Price', 'Selling Price', 'Quantity',
       'Model Number', 'About Product', 'Product Specification',
       'Technical Details', 'Shipping Weight', 'Product Dimensions', 'Image',
       'Variants', 'Sku', 'Product Url', 'Stock', 'Product Details',
       'Dimensions', 'Color', 'Ingredients', 'Direction To Use',
       'Is Amazon Seller', 'Size Quantity Variant', 'Product Description'],
      dtype='object')

In [6]:
# Deleting unwanted columns
del_list = ['Uniq Id', 'Brand Name', 'Asin', 'Category', 'Upc Ean Code', 'List Price', 'Selling Price', 'Quantity', 
            'Model Number', 'Product Specification', 'Technical Details', 'Shipping Weight', 'Product Dimensions', 'Variants',
            'Sku', 'Product Url', 'Stock', 'Product Details', 'Dimensions', 'Color', 'Ingredients', 'Direction To Use', 
            'Is Amazon Seller', 'Size Quantity Variant', 'Product Description']

fdata = fdata.drop(del_list, axis = 1)

fdata.columns

Index(['Product Name', 'About Product', 'Image'], dtype='object')

In [7]:
def clear_text(text):
    clear_content = re.sub(r'[\t\r\n]', ' ', text)
    clear_content = re.sub(' +',' ', clear_content).lstrip()

    return clear_content

In [8]:
def insert_product(row):   
    encoded_string = row['About Product'].encode('utf-8', "ignore")
    description = encoded_string.decode()
    
    description = description.replace("Make sure this fits by entering your model number. | ", "")
    description = re.sub('[👍🌍📱✅【】👐🔬⚗️⚙️🧜😎🔴🔵🙌🌿🎁🦄💃👜⭐💦]', '', description)
    description = clear_text(description.replace(' | ', ''))
    
    query = "INSERT INTO products_tmp_amazon (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
    values = (row['Product Name'], description, "Amazon - 2020", "f")

    cursor.execute(query, values)
    conn.commit()

    return cursor.lastrowid

In [9]:
for index, row in fdata.iterrows():
    if row['Product Name'] == "NaN" or type(row['Product Name']) is not str: row['Product Name'] = ""
    if row['About Product'] == "NaN" or type(row['About Product']) is not str: row['About Product'] = ""

    if row['Product Name'] is not "" and row['About Product'] is not "":
        product_id = insert_product(row)

        image = row['Image'].split(sep="|")[0]

        folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}.jpeg")

        try:
            file = urllib.request.urlretrieve(image, folder_dest)
        except:
            print(f"Product {product_id}: Error retrieving image ({image})")

Product 450: Error retrieving image (https://m.media-amazon.com/images/I/A13usaonutL.jpg)
Product 776: Error retrieving image (https://m.media-amazon.com/images/I/A13usaonutL.jpg)
Product 2139: Error retrieving image (https://m.media-amazon.com/images/I/A13usaonutL.jpg)
Product 3393: Error retrieving image (https://m.media-amazon.com/images/I/A1nYNISnPeL.jpg)
Product 3478: Error retrieving image (https://m.media-amazon.com/images/I/A13usaonutL.jpg)
Product 3661: Error retrieving image (https://m.media-amazon.com/images/I/A13usaonutL.jpg)
Product 4813: Error retrieving image (https://images-na.ssl-images-amazon.com/images/I/71Q33N6MEVL.jpg)
Product 5174: Error retrieving image (https://m.media-amazon.com/images/I/B1qmQK-r4OS.jpg)
Product 5258: Error retrieving image (https://m.media-amazon.com/images/I/A13usaonutL.jpg)
Product 5988: Error retrieving image (https://m.media-amazon.com/images/I/A13usaonutL.jpg)
Product 6380: Error retrieving image (https://m.media-amazon.com/images/I/A13us